In [1]:
from datetime import date
from portfolio_rl.impact.impact_models import ACImpactModel, OWImpactModel
import numpy as np

### Almgren–Chriss model

In [3]:
ac = ACImpactModel()

In [4]:
ac.get_impact_history()

,date,symbol,permanent_impact


In [5]:
ac.apply_trade(100, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=1.2, price_shift=6.000000000000001e-09)

In [6]:
ac.apply_trade(10e3, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=1200.0, price_shift=6.000000000000002e-07)

In [7]:
ac.apply_trade(50e3, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=13416.407864998737, price_shift=3e-06)

In [8]:
ac.end_day(date(2025, 1, 1))

In [9]:
ac.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004


In [10]:
ac.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.33192202055, price_shift=6e-06)

In [11]:
ac.end_day(date(2025, 1, 2))

In [12]:
ac.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004
1,2025-01-02,nvda,0.000010


In [13]:
ac.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.33192202055, price_shift=6e-06)

In [14]:
ac.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.33192202055, price_shift=6e-06)

In [15]:
ac.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.33192202055, price_shift=6e-06)

In [16]:
ac.end_day(date(2025, 1, 3))

In [17]:
ac.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004
1,2025-01-02,nvda,0.000010
2,2025-01-03,nvda,0.000028


---

### Obizhaeva–Wang model

In [20]:
ow = OWImpactModel()

In [21]:
ow.get_impact_history()

,date,symbol,permanent_impact


In [22]:
ow.apply_trade(100, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=1.2, price_shift=6.000000000000001e-09)

In [23]:
ow.apply_trade(10e3, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=1200.0000000014647, price_shift=6.000000000000002e-07)

In [24]:
ow.apply_trade(50e3, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=13416.407865326286, price_shift=3e-06)

In [25]:
ow.end_day(date(2025, 1, 1))

In [26]:
ow.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004


In [27]:
ow.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.33192433673, price_shift=6e-06)

In [28]:
ow.end_day(date(2025, 1, 2))

In [29]:
ow.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004
1,2025-01-02,nvda,0.000010


In [30]:
ow.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.331926653074, price_shift=6e-06)

In [31]:
ow.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.331926653365, price_shift=6e-06)

In [32]:
ow.apply_trade(1e5, 100, 0.02, 1e6, 'nvda')

TradeImpact(cost=37947.331926653365, price_shift=6e-06)

In [33]:
ow.end_day(date(2025, 1, 3))

In [34]:
ow.get_impact_history()

,date,symbol,permanent_impact
0,2025-01-01,nvda,0.000004
1,2025-01-02,nvda,0.000010
2,2025-01-03,nvda,0.000028


---

## AC x OW diff

In [3]:
ow = OWImpactModel(half_life_minutes=30)
ac = ACImpactModel()

x = 100_000
volatility = 0.02
volume = 1_000_000
price = 100

total_cost_ow = 0
total_cost_ac = 0

trade_interval_minutes = 30
decay_per_trade = np.exp(-ow.kappa * (trade_interval_minutes / 390))

for i in range(10):
    # OW with decay between trades
    total_cost_ow += ow.apply_trade(x, price, volatility, volume, 'NVDA').cost
    ow._transient_states['NVDA'] *= decay_per_trade

    # AC model
    total_cost_ac += ac.apply_trade(x, price, volatility, volume, 'NVDA').cost

print(f"OW total cost (with proper transient impact): {total_cost_ow:.2f}")
print(f"AC total cost: {total_cost_ac:.2f}")
print(f"Difference (OW - AC): {total_cost_ow - total_cost_ac:.2f}")

OW total cost (with proper transient impact): 379494.17
AC total cost: 379473.32
Difference (OW - AC): 20.85
